In [1]:
import pandas as pd

import numpy as np
import gc

In [3]:
used_cols = ['item_id', 'user_id', 'city', 'region', 'category_name',
             'price','title','description', 'item_seq_number',
             'activation_date', 'user_type']

train = pd.read_csv('../input/train.csv', usecols=used_cols)
train_active = pd.read_csv('../input/train_active.csv', usecols=used_cols)
test = pd.read_csv('../input/test.csv', usecols=used_cols)
test_active = pd.read_csv('../input/test_active.csv', usecols=used_cols)

train_periods = pd.read_csv('../input/periods_train.csv', 
                            parse_dates=['activation_date', 'date_from', 'date_to'])
test_periods = pd.read_csv('../input/periods_test.csv', 
                           parse_dates=['activation_date', 'date_from', 'date_to'])

train.head()


,item_id,user_id,region,city,category_name,title,description,price,item_seq_number,activation_date,user_type
0,b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Товары для детей и игрушки,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2,2017-03-28,Private
1,2dac0150717d,39aeb48f0017,Самарская область,Самара,Мебель и интерьер,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19,2017-03-26,Private
2,ba83aefab5dc,91e2f88dd6e3,Ростовская область,Ростов-на-Дону,Аудио и видео,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,9,2017-03-20,Private
3,02996f1dd2ea,bf5cccea572d,Татарстан,Набережные Челны,Товары для детей и игрушки,Автокресло,Продам кресло от0-25кг,2200.0,286,2017-03-25,Company
4,7c90be56d2ab,ef50846afc0b,Волгоградская область,Волгоград,Автомобили,"ВАЗ 2110, 2003",Все вопросы по телефону.,40000.0,3,2017-03-16,Private


In [4]:
def process_df(df):
    df['title_len'] = df['title'].fillna('').apply(lambda x:len(x))
    df['description_len'] = df['description'].fillna('').apply(lambda x:len(x))
    df['price_log'] = df['price'].apply(lambda x: np.log(x+0.01))
    return df[['item_id', 'user_id', 'city', 'region', 'category_name',
               'price_log','title_len','description_len','item_seq_number',
             'activation_date', 'user_type']]

In [5]:
all_samples = pd.concat([
    process_df(train),
    process_df(train_active),
    process_df(test),
    process_df(test_active)
]).reset_index(drop=True)
all_samples.drop_duplicates(['item_id'], inplace=True)

del train_active
del test_active
gc.collect()

49

In [6]:
def do_count( df, group_cols, agg_type='uint32', show_max=False, show_agg=True ):
    agg_name='{}count'.format('_'.join(group_cols))  
    if show_agg:
        print( "\nAggregating by ", group_cols ,  '... and saved in', agg_name )
    gp = df[group_cols][group_cols].groupby(group_cols).size().rename(agg_name).\
            to_frame().reset_index()
    df = df.merge(gp, on=group_cols, how='left')
    del gp
    if show_max:
        print( agg_name + " max value = ", df[agg_name].max() )
    df[agg_name] = df[agg_name].astype(agg_type)
    
#     print('predictors',predictors)
    gc.collect()
    return( df )

##  Below a function is written to extract unique count feature from different cols
def do_countuniq( df, group_cols, counted, agg_type='uint32', show_max=False, show_agg=True ):
    agg_name= '{}_by_{}_countuniq'.format(('_'.join(group_cols)),(counted))  
    if show_agg:
        print( "\nCounting unqiue ", counted, " by ", group_cols ,  '... and saved in', agg_name )
    gp = df[group_cols+[counted]].groupby(group_cols)[counted].nunique().reset_index().rename(columns={counted:agg_name})
    df = df.merge(gp, on=group_cols, how='left')
    del gp
    if show_max:
        print( agg_name + " max value = ", df[agg_name].max() )
    df[agg_name] = df[agg_name].astype(agg_type)
    
#     print('predictors',predictors)
    gc.collect()
    return( df )

### Below a function is written to extract mean feature  from different cols
def do_mean( df, group_cols, counted, agg_type='float32', show_max=False, show_agg=True ):
    agg_name= '{}_by_{}_mean'.format(('_'.join(group_cols)),(counted))  
    if show_agg:
        print( "\nCalculating mean of ", counted, " by ", group_cols , '... and saved in', agg_name )
    gp = df[group_cols+[counted]].groupby(group_cols)[counted].mean().reset_index().rename(columns={counted:agg_name})
    df = df.merge(gp, on=group_cols, how='left')
    del gp
    if show_max:
        print( agg_name + " max value = ", df[agg_name].max() )
    df[agg_name] = df[agg_name].astype(agg_type)
    
#     print('predictors',predictors)
    gc.collect()
    return( df )

def do_var( df, group_cols, counted, agg_type='float32', show_max=False, show_agg=True ):
    agg_name= '{}_by_{}_var'.format(('_'.join(group_cols)),(counted)) 
    if show_agg:
        print( "\nCalculating variance of ", counted, " by ", group_cols , '... and saved in', agg_name )
    gp = df[group_cols+[counted]].groupby(group_cols)[counted].var().reset_index().rename(columns={counted:agg_name})
    df = df.merge(gp, on=group_cols, how='left')
    del gp
    if show_max:
        print( agg_name + " max value = ", df[agg_name].max() )
    df[agg_name] = df[agg_name].astype(agg_type)
#     print('predictors',predictors)
    gc.collect()
    return( df )

In [7]:
all_samples['item_seq_number'] = all_samples.item_seq_number.fillna(-1)\
.apply(lambda x:int(x))

In [8]:
all_samples['city'] = all_samples.city.fillna('NA')
all_samples['user_type'] = all_samples.user_type.fillna('NA')
all_samples['activation_date'] = all_samples.activation_date.fillna('NA')


In [9]:
for gpby in ['city','activation_date','user_type']:
    all_samples = do_countuniq(all_samples, [gpby],'item_seq_number')
    all_samples = do_countuniq(all_samples, [gpby],'user_id')
    all_samples = do_countuniq(all_samples, [gpby],'category_name')
    all_samples = do_count(all_samples, [gpby])
    all_samples = do_mean(all_samples, [gpby], 'title_len')
    all_samples = do_mean(all_samples, [gpby], 'price_log')
    all_samples = do_mean(all_samples, [gpby], 'description_len')
    all_samples = do_var(all_samples, [gpby], 'title_len')
    all_samples = do_var(all_samples, [gpby], 'price_log')
    all_samples = do_var(all_samples, [gpby], 'description_len')
    



Counting unqiue  item_seq_number  by  ['city'] ... and saved in city_by_item_seq_number_countuniq

Counting unqiue  user_id  by  ['city'] ... and saved in city_by_user_id_countuniq

Counting unqiue  category_name  by  ['city'] ... and saved in city_by_category_name_countuniq

Aggregating by  ['city'] ... and saved in citycount

Calculating mean of  title_len  by  ['city'] ... and saved in city_by_title_len_mean

Calculating mean of  price_log  by  ['city'] ... and saved in city_by_price_log_mean

Calculating mean of  description_len  by  ['city'] ... and saved in city_by_description_len_mean

Calculating variance of  title_len  by  ['city'] ... and saved in city_by_title_len_var

Calculating variance of  price_log  by  ['city'] ... and saved in city_by_price_log_var

Calculating variance of  description_len  by  ['city'] ... and saved in city_by_description_len_var

Counting unqiue  item_seq_number  by  ['activation_date'] ... and saved in activation_date_by_item_seq_number_countuniq


In [ ]:
gpby = 'item_seq_number'
all_samples = do_countuniq(all_samples, [gpby],'user_id')
all_samples = do_countuniq(all_samples, [gpby],'category_name')
all_samples = do_countuniq(all_samples, [gpby],'city')
all_samples = do_count(all_samples, [gpby])
all_samples = do_mean(all_samples, [gpby], 'title_len')
all_samples = do_mean(all_samples, [gpby], 'price_log')
all_samples = do_mean(all_samples, [gpby], 'description_len')
all_samples = do_var(all_samples, [gpby], 'title_len')
all_samples = do_var(all_samples, [gpby], 'price_log')
all_samples = do_var(all_samples, [gpby], 'description_len')


Counting unqiue  user_id  by  ['item_seq_number'] ... and saved in item_seq_number_by_user_id_countuniq


In [ ]:
# get city info
df_city = all_samples[[x for x in all_samples.columns if 'city' in x[:5]]]
df_city.drop_duplicates(['city'],inplace=True)
df_city.to_csv('city_features.csv', index=False)
del df_city; gc.collect()

In [ ]:
# get activate
df_actdate = all_samples[[x for x in all_samples.columns if 'activat' in x[:10]]]
df_actdate.drop_duplicates(['activation_date'],inplace=True)
df_actdate.to_csv('activation_date_features.csv', index=False)
del df_actdate; gc.collect()

In [ ]:
# get usertype
df_usrtp = all_samples[[x for x in all_samples.columns if 'user_type' in x[:10]]]
df_usrtp.drop_duplicates(['user_type'],inplace=True)
df_usrtp.to_csv('user_type_features.csv', index=False)
del df_usrtp; gc.collect()

In [ ]:
# get item_seq_number
df_itsqn = all_samples[[x for x in all_samples.columns if 'item_seq_number' in x[:20]]]
df_itsqn.drop_duplicates(['item_seq_number'],inplace=True)
df_itsqn.to_csv('item_seq_number_features.csv', index=False)
del df_itsqn; gc.collect()

In [ ]:
all_samples.head()

In [ ]:
all_samples.drop(['item_seq_number','activation_date', 'user_type'],
                axis=1,inplace=True)

In [ ]:
all_periods = pd.concat([
    train_periods,
    test_periods
])

del train_periods
del test_periods
gc.collect()

all_periods['date_from_isweekend'] = all_periods['date_from'].dt.dayofweek >= 5
all_periods['date_to_isweekend'] = all_periods['date_to'].dt.dayofweek >= 5
all_periods['activation_date_isweekend'] = all_periods['activation_date'].dt.dayofweek >= 5
all_periods['days_up'] = all_periods['date_to'].dt.dayofyear - all_periods['date_from'].dt.dayofyear

all_periods.head()

In [ ]:
all_periods['date_from_isweekend'] = all_periods['date_from'].dt.dayofweek >= 5
all_periods['date_to_isweekend'] = all_periods['date_to'].dt.dayofweek >= 5
all_periods['activation_date_isweekend'] = all_periods['activation_date'].dt.dayofweek >= 5

In [ ]:
gp = all_periods.groupby(['item_id'])[['days_up']]
gp2 = all_periods.groupby(['item_id'])[[
    'date_from_isweekend','date_to_isweekend','activation_date_isweekend']]

gp_df = pd.DataFrame()
gp_df['days_up_sum'] = gp.sum()['days_up']
gp_df['times_put_up'] = gp.count()['days_up']
gp_df[['date_from_isweekend','date_to_isweekend','activation_date_isweekend']] = \
    gp2.mean()
gp_df.reset_index(inplace=True)
gp_df.rename(index=str, columns={'index': 'item_id'})

gp_df.head()


In [ ]:
all_periods = all_periods.drop(
    ['date_from_isweekend','date_to_isweekend','activation_date_isweekend'],
    axis=1)
all_periods.drop_duplicates(['item_id'], inplace=True)
all_periods = all_periods.merge(gp_df, on='item_id', how='left')
all_periods.head()

In [ ]:
all_periods = all_periods.merge(all_samples, on='item_id', how='left')
all_periods.head()

In [ ]:
gp = all_periods.groupby(['user_id'])[
    ['days_up_sum', 
     'times_put_up',
     'date_from_isweekend',
     'date_to_isweekend',
     'activation_date_isweekend',
     'price_log',
     'title_len',
     'description_len'
     ]
].mean().reset_index() \
    .rename(index=str, columns={
        'days_up_sum': 'avg_days_up_user',
        'times_put_up': 'avg_times_up_user',
        'date_from_isweekend': 'avg_date_from_isweekend',
        'date_to_isweekend': 'avg_date_to_isweekend',
        'activation_date_isweekend': 'avg_activation_date_isweekend',
        'price_log': 'avg_price_log',
        'title_len': 'avg_title_len',
        'description_len': 'avg_description_len'
    })
gp.head()



In [ ]:
gp2 = all_periods.groupby(['user_id'])[
    ['city', 
     'region',
     'category_name'
     ]
].nunique().reset_index() \
    .rename(index=str, columns={
        'city': 'nunique_city',
        'region': 'nunique_region',
        'category_name': 'nunique_category_name'
    })
gp2.head()

In [ ]:
gp = gp.merge(gp2,on='user_id')

In [ ]:
n_user_items = all_samples.groupby(['user_id'])[['item_id']].count().reset_index() \
    .rename(index=str, columns={
        'item_id': 'n_user_items'
    })
gp = gp.merge(n_user_items, on='user_id', how='outer')

gp.head()

In [ ]:
gp.describe()

In [ ]:
gp.to_csv('aggregated_features_new.csv', index=False)

